In [55]:
import pandas as pd

Something is not right here, take a look at the source code in penguin:

https://github.com/raphaelvallat/pingouin/blob/main/src/pingouin/reliability.py

relies on ANOVA

https://github.com/raphaelvallat/pingouin/blob/main/src/pingouin/parametric.py


In [67]:
path = "../data/appraisal/export_appraisal.csv"
df = pd.read_csv(path)
df.head()

,user_id,survey_id,survey_item_index,filename,has_reply,emotion_1,video_id,mix,emotion_1_id,emotion_2_id,intensity_level,reply_dim_Novelty,reply_dim_Pleasantness,reply_dim_Goal conduciveness,reply_dim_Urgency,reply_dim_Power,reply_dim_Norm compatibility
0,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,0,A200_int_v_3.mp4,1,interest_curiosity,A200,0,29,100,3.0,1,4,4,1,3,3
1,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,1,A102_exc_v_3.mp4,1,excitement_expectation,A102,0,36,100,3.0,1,5,5,1,3,5
2,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,2,A102_fea_p_2.mp4,1,fear,A102,0,10,100,2.0,3,1,1,2,2,1
3,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,3,A102_amu_v_3.mp4,1,amusement,A102,0,13,100,3.0,2,4,4,2,3,4
4,default,1d1439eb990b9fbdc99bf899dc7a7f9cb61a9a404e5677...,4,A207_pea_v_2.mp4,1,peacefulness_serenity,A207,0,25,100,2.0,1,2,2,2,2,2


In [57]:
scales = [
    'reply_dim_Novelty',
    'reply_dim_Pleasantness',
    'reply_dim_Goal conduciveness',
    'reply_dim_Urgency',
    'reply_dim_Power',
    'reply_dim_Norm compatibility'
]

scales_df = df[scales]
scales_df

,reply_dim_Novelty,reply_dim_Pleasantness,reply_dim_Goal conduciveness,reply_dim_Urgency,reply_dim_Power,reply_dim_Norm compatibility
0,1,4,4,1,3,3
1,1,5,5,1,3,5
2,3,1,1,2,2,1
3,2,4,4,2,3,4
4,1,2,2,2,2,2
...,...,...,...,...,...,...
787,3,5,5,3,4,5
788,5,1,1,4,4,2
789,4,5,5,4,5,5
790,1,3,2,2,4,3


# One-way random effects, absolute agreement, single rater/measurement

in the context of Intraclass Correlation Coefficient (ICC) calculations and particularly in Analysis of Variance (ANOVA) frameworks, "Mean Square for Rows" (MSR) can correspond to what is often referred to as "Mean Square Between" (MSB). This terminology comes from the layout of the ANOVA table, where different sources of variation are partitioned, and the "rows" in this case refer to the different groups or categories being compared — in this scenario, the different videos.

In [58]:
scale = scales[0]

### MSB for videos

In [59]:
video_means = df.groupby('filename')[scale].mean()

grand_mean = df[scales[0]].mean()

# Compute the squared differences from the grand mean for each video's mean rating
squared_diffs = (video_means - grand_mean) ** 2

# Compute MSB: mean of these squared differences, adjust for degrees of freedom if necessary
MSB = squared_diffs.mean()
MSB

0.6983471074380164

### MSE for videos

In [60]:
intermediary_df = df[['filename', scale]].copy()
video_means = intermediary_df.groupby('filename')[scale].mean().rename('video_mean')
intermediary_df = intermediary_df.merge(video_means, on='filename')

intermediary_df['squared_diffs'] = (intermediary_df[scale] - intermediary_df['video_mean']) ** 2

MSE = intermediary_df['squared_diffs'].mean()
MSE

1.0364758698092031

### MSE for videos with rater adjustment

In [61]:
# Assume 'df' is your original DataFrame, and 'scale' is the column with the ratings
# Copy the DataFrame for calculations
df_calc = df.copy()

# Calculate the mean rating for each video
video_means = df.groupby('filename')[scale].mean().rename('video_mean')

# Calculate the overall mean rating for each rater
rater_means = df.groupby('survey_id')[scale].mean().rename('rater_mean')

# Merge these means into the copy of the DataFrame
df_calc = df_calc.merge(video_means, on='filename')
df_calc = df_calc.merge(rater_means, on='survey_id')

# Adjust ratings by removing the rater's mean and adding the global mean
df_calc['adjusted_rating'] = df_calc[scale] - df_calc['rater_mean'] + df[scale].mean()

# Calculate the squared deviations of adjusted ratings from each video's mean rating
df_calc['squared_dev'] = (df_calc['adjusted_rating'] - df_calc['video_mean']) ** 2

# Calculate MSE, accounting for degrees of freedom: total ratings minus number of videos and raters
MSE_adjusted = df_calc['squared_dev'].sum() / (len(df_calc) - df_calc['filename'].nunique() - df_calc['survey_id'].nunique() + 1)

# MSE_adjusted is the result and doesn't affect the original df
print("MSE (adjusted for rater bias):", MSE_adjusted)

MSE (adjusted for rater bias): 0.8790164576802507


In [62]:
k = df['survey_id'].nunique()
ICC_3_1 = (MSB - MSE_adjusted) / (MSB +  (k -1) * MSE_adjusted)
ICC_3_1

-0.023371042809087146

### MSB for emotions

In [63]:
# Calculate the grand mean across all ratings
grand_mean = df[scales[0]].mean()

# Calculate the mean rating for each construct
construct_means = df.groupby('emotion_1')[scale].mean()

# Compute the squared differences from the grand mean for each construct's mean rating
squared_diffs_constructs = (construct_means - grand_mean) ** 2

# Compute MSB for each construct: mean of these squared differences
# Adjust for degrees of freedom if necessary. Here, it's just the mean, but in a full ANOVA, you'd divide by the df between constructs
MSB_constructs = squared_diffs_constructs.mean()

MSB_constructs

0.6001428425670849

### MSE for emotions

In [64]:
intermediary_df = df[['emotion_1', scale]].copy()
video_means = intermediary_df.groupby('emotion_1')[scale].mean().rename('emotion_mean')
intermediary_df = intermediary_df.merge(video_means, on='emotion_1')

intermediary_df['squared_diffs'] = (intermediary_df[scale] - intermediary_df['emotion_mean']) ** 2

MSE = intermediary_df['squared_diffs'].mean()
MSE

1.1346801346801347

In [65]:
k = df['survey_id'].nunique()
ICC_3_1 = (MSB - MSE) / (MSB +  (k -1) * MSE_adjusted)
ICC_3_1

-0.056443208795634976